In [139]:
import uuid
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import my_connector
from datetime import datetime
from my_connector import MySQLConnector

# Limpieza

In [140]:
df = pd.read_csv("Datasets/padron22.csv", low_memory=False, encoding='UTF-8')

In [141]:
df.head()

,Ï»¿EJERICIO,COD_TIPO_PERSONA,TIPO_PERSONA,COD_DISTRITO,DISTRITO,COD_BARRIO,BARRIO,COD_TIPO_VEHICULO,TIPO_VEHICULO,ETIQUETA_MEDIOAMBIENTAL,CLASIFICACIÃN_AMBIENTAL,CUOTA,TIPO_CARBURANTE,AÃO_MATRICULACIÃN,CONTADOR
0,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,Tipo carburante desconocido,2013,1
1,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2006,1
2,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2007,1
3,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2009,1
4,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2015,1


In [142]:
df.head(20)

,Ï»¿EJERICIO,COD_TIPO_PERSONA,TIPO_PERSONA,COD_DISTRITO,DISTRITO,COD_BARRIO,BARRIO,COD_TIPO_VEHICULO,TIPO_VEHICULO,ETIQUETA_MEDIOAMBIENTAL,CLASIFICACIÃN_AMBIENTAL,CUOTA,TIPO_CARBURANTE,AÃO_MATRICULACIÃN,CONTADOR
0,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,Tipo carburante desconocido,2013,1
1,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2006,1
2,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2007,1
3,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2009,1
4,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2015,1
5,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificaciÃ³n ambiental,149.0,DIESEL,2010,1
6,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,B,Etiqueta B,73.0,DIESEL,2015,1
7,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,B,Etiqueta B,149.0,DIESEL,2014,1
8,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,C,Etiqueta C,73.0,DIESEL,2018,2
9,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,C,Etiqueta C,149.0,DIESEL,2019,3


Tiramos las columnas que no podemos utilizar para cruzar con accidentalidad. Así como los datos repetidos

In [143]:
df.drop(columns=['Ï»¿EJERICIO','DISTRITO', 'BARRIO', 'COD_BARRIO', 'CONTADOR'], inplace=True)

Arreglamos los códigos para que tengan sentido

In [144]:
cod_veh = df['COD_TIPO_VEHICULO'].unique()
tipo_veh = df['TIPO_VEHICULO'].unique()

In [145]:
cod_veh

array(['CA', 'MT', 'RE', 'TR', 'TU', 'CI', 'AU', 'SE'], dtype=object)

In [146]:
tipo_veh

array(['CAMION', 'MOTOCICLETA', 'OTROS', 'TURISMO'], dtype=object)

In [147]:
for row in zip(cod_veh, tipo_veh):
    print(row)

('CA', 'CAMION')
('MT', 'MOTOCICLETA')
('RE', 'OTROS')
('TR', 'TURISMO')


In [148]:
df['COD_TIPO_VEHICULO'] = df['COD_TIPO_VEHICULO'].replace({'CA': 'C', 'MT': 'M', 'RE': 'O', 'TR': 'T'})

In [149]:
cod_veh = df['COD_TIPO_VEHICULO'].unique()
tipo_veh = df['TIPO_VEHICULO'].unique()
for row in zip(cod_veh, tipo_veh):
    print(row)

('C', 'CAMION')
('M', 'MOTOCICLETA')
('O', 'OTROS')
('T', 'TURISMO')


In [150]:
df.pop('TIPO_VEHICULO').head()

0    CAMION
1    CAMION
2    CAMION
3    CAMION
4    CAMION
Name: TIPO_VEHICULO, dtype: object

In [151]:
df.head()

,COD_TIPO_PERSONA,TIPO_PERSONA,COD_DISTRITO,COD_TIPO_VEHICULO,ETIQUETA_MEDIOAMBIENTAL,CLASIFICACIÃN_AMBIENTAL,CUOTA,TIPO_CARBURANTE,AÃO_MATRICULACIÃN
0,E,Ente sin Personalidad Art 35.4,0,C,--,Sin clasificaciÃ³n ambiental,73.0,Tipo carburante desconocido,2013
1,E,Ente sin Personalidad Art 35.4,0,C,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2006
2,E,Ente sin Personalidad Art 35.4,0,C,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2007
3,E,Ente sin Personalidad Art 35.4,0,C,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2009
4,E,Ente sin Personalidad Art 35.4,0,C,--,Sin clasificaciÃ³n ambiental,73.0,DIESEL,2015


In [152]:
cod_veh = df['ETIQUETA_MEDIOAMBIENTAL'].unique()
tipo_veh = df['CLASIFICACIÃ\x93N_AMBIENTAL'].unique()
for row in zip(cod_veh, tipo_veh):
    print(row)

('--', 'Sin clasificaciÃ³n ambiental')
('B', 'Etiqueta B')
('C', 'Etiqueta C')
(nan, 'Sin distintivo Ambiental')
('0', 'Cero Emisiones')
('E', 'ECO')


In [153]:
df['ETIQUETA_MEDIOAMBIENTAL'] = df['ETIQUETA_MEDIOAMBIENTAL'].replace({'--': 'A'})
df['ETIQUETA_MEDIOAMBIENTAL'] = df['ETIQUETA_MEDIOAMBIENTAL'].fillna('A')
df.pop('CLASIFICACIÃ\x93N_AMBIENTAL').head()

0    Sin clasificaciÃ³n ambiental
1    Sin clasificaciÃ³n ambiental
2    Sin clasificaciÃ³n ambiental
3    Sin clasificaciÃ³n ambiental
4    Sin clasificaciÃ³n ambiental
Name: CLASIFICACIÃN_AMBIENTAL, dtype: object

In [154]:
df['COD_TIPO_PERSONA'].unique()

array(['E', 'F', 'J', 'O'], dtype=object)

In [155]:
df['COD_TIPO_PERSONA'] = df['COD_TIPO_PERSONA'].replace({'E': 'S'})

Solo existe una fila con la O y no tiene una clasificación clara así que lo he tirado

In [156]:
df = df.drop(df[df['COD_TIPO_PERSONA'] == 'O'].index)

In [157]:
df.pop('TIPO_PERSONA').head()

0    Ente sin Personalidad Art 35.4
1    Ente sin Personalidad Art 35.4
2    Ente sin Personalidad Art 35.4
3    Ente sin Personalidad Art 35.4
4    Ente sin Personalidad Art 35.4
Name: TIPO_PERSONA, dtype: object

In [158]:
df[df['COD_DISTRITO']==0]

,COD_TIPO_PERSONA,COD_DISTRITO,COD_TIPO_VEHICULO,ETIQUETA_MEDIOAMBIENTAL,CUOTA,TIPO_CARBURANTE,AÃO_MATRICULACIÃN
0,S,0,C,A,73.0,Tipo carburante desconocido,2013
1,S,0,C,A,73.0,DIESEL,2006
2,S,0,C,A,73.0,DIESEL,2007
3,S,0,C,A,73.0,DIESEL,2009
4,S,0,C,A,73.0,DIESEL,2015
...,...,...,...,...,...,...,...
156973,J,0,TU,A,224.0,GASOLINA,1999
156974,J,0,TU,A,224.0,GASOLINA,2005
156975,J,0,TU,A,224.0,GASOLINA,2007
156976,J,0,TU,A,224.0,GASOLINA,2018


In [159]:
df['TIPO_CARBURANTE'].unique()

array(['Tipo carburante desconocido', 'DIESEL', 'GASOLINA', 'OTROS',
       'HIBRIDO ENCHUFABLES PHEV', 'HÃ\x8dBRIDO',
       'GAS LICUADO DE PETROLEO', 'GAS NATURAL COMPRIMIDO',
       'ELÃ\x89CTRICO', 'ELECTRICOS RANGO EXTENDIDO',
       'GAS NATURAL LICUADO', 'ETANOL', 'BIOMETANO', 'BIODIESEL'],
      dtype=object)

In [160]:
df['TIPO_CARBURANTE'] = df['TIPO_CARBURANTE'].replace({'HÃBRIDO': 'HIBRIDO', 'ELÃCTRICO': 'ELECTRICO', 'Tipo carburante desconocido':'DESCONOCIDO'})

In [163]:
df.rename(columns={'AÃ\x91O_MATRICULACIÃ\x93N':'Y_MATRICULACION'}, inplace=True)

In [164]:
df

,COD_TIPO_PERSONA,COD_DISTRITO,COD_TIPO_VEHICULO,ETIQUETA_MEDIOAMBIENTAL,CUOTA,TIPO_CARBURANTE,Y_MATRICULACION
0,S,0,C,A,73.0,DESCONOCIDO,2013
1,S,0,C,A,73.0,DIESEL,2006
2,S,0,C,A,73.0,DIESEL,2007
3,S,0,C,A,73.0,DIESEL,2009
4,S,0,C,A,73.0,DIESEL,2015
...,...,...,...,...,...,...,...
180340,J,21,TU,A,129.0,DIESEL,2001
180341,J,21,TU,A,129.0,DIESEL,2003
180342,J,21,TU,A,129.0,DIESEL,2005
180343,J,21,TU,A,129.0,GASOLINA,2000


Vamos a clusterizar los datos ya que hay muchos repetidos

In [168]:
df.groupby(['COD_DISTRITO']).count()

,COD_TIPO_PERSONA,COD_TIPO_VEHICULO,ETIQUETA_MEDIOAMBIENTAL,CUOTA,TIPO_CARBURANTE,Y_MATRICULACION
COD_DISTRITO,,,,,,
0,5039,5039,5039,5039,5039,5039
1,8355,8355,8355,8355,8355,8355
2,8277,8277,8277,8277,8277,8277
3,8318,8318,8318,8318,8318,8318
4,10520,10520,10520,10520,10520,10520
5,11121,11121,11121,11121,11121,11121
6,9161,9161,9161,9161,9161,9161
7,9520,9520,9520,9520,9520,9520
8,11171,11171,11171,11171,11171,11171
